In [59]:
import pandas as pd
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn as dglnn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import pickle
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dgl.function as fn
from dgl.nn import GraphConv, GATConv
import dgl
import dgl.nn as dglnn
import pickle


In [60]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import dgl

class BPRDataset(Dataset):
    def __init__(self, user_item_pairs, total_items):
        self.user_item_pairs = user_item_pairs
        self.total_items = total_items

    def __len__(self):
        return len(self.user_item_pairs)

    def __getitem__(self, idx):
        user, pos_item = self.user_item_pairs[idx]
        neg_item = np.random.randint(0, self.total_items)
        while neg_item == pos_item:
            neg_item = np.random.randint(0, self.total_items)
        return user, pos_item, neg_item




graph_path = r'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\mainmodel\graph\hetero_graph03_with_V_T_A_features.pkl'

with open(graph_path, 'rb') as f:
    graph = pickle.load(f)
data_path = 'D:\\CODE\\multi-model knowledge graph multi-graph recommendation system\\data\\cleanuser_rating.csv'
df = pd.read_csv(data_path)
total_items = df['movieId'].max() + 1  
print("Total items (movies):", total_items)

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df['userId'] = user_encoder.fit_transform(df['userId'])
df['movieId'] = item_encoder.fit_transform(df['movieId'])
num_users = df['userId'].nunique() 
num_items = df['movieId'].nunique()  
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

train_pairs = list(zip(train_df['userId'].values, train_df['movieId'].values))
valid_pairs = list(zip(valid_df['userId'].values, valid_df['movieId'].values))

total_items = df['movieId'].max() + 1

train_dataset = BPRDataset(train_pairs, total_items)
valid_dataset = BPRDataset(valid_pairs, total_items)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)


Total items (movies): 131263


In [61]:
class EnhancedHeteroGraphConv(nn.Module):
    def __init__(self, num_users, num_items, feature_sizes, embedding_dim=128, id_embedding_size=16, num_heads=4):
        super(EnhancedHeteroGraphConv, self).__init__()

        # 用户和项目的嵌入层
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)

        # 基于ID的节点的嵌入层
        self.embeddings = nn.ModuleDict({
            node_type: nn.Embedding(10000, id_embedding_size) for node_type in feature_sizes if node_type not in ['movie', 'movieimage', 'movietext', 'movieaudio']
        })

        # 特定模态特征的转换层
        self.image_transform = nn.Linear(feature_sizes['movieimage'], 32)
        self.text_transform = nn.Linear(feature_sizes['movietext'], 32)
        self.audio_transform = nn.Linear(feature_sizes['movieaudio'], 32)

        # 每种模态的GAT层
        self.gat_image1 = GATConv(32, 32, num_heads=num_heads, allow_zero_in_degree=True)
        self.gat_image2 = GATConv(32 * num_heads, 32, num_heads=num_heads, allow_zero_in_degree=True)

        self.gat_text1 = GATConv(32, 32, num_heads=num_heads, allow_zero_in_degree=True)
        self.gat_text2 = GATConv(32 * num_heads, 32, num_heads=num_heads, allow_zero_in_degree=True)

        self.gat_audio1 = GATConv(32, 32, num_heads=num_heads, allow_zero_in_degree=True)
        self.gat_audio2 = GATConv(32 * num_heads, 32, num_heads=num_heads, allow_zero_in_degree=True)

        # 异构图卷积层
        self.hetero_conv = dgl.nn.HeteroGraphConv({
            ('movie', 'has_actor', 'actor'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('movie', 'has_actress', 'actress'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('movie', 'has_composer', 'composer'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('movie', 'has_director', 'director'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('movie', 'has_editor', 'editor'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('movie', 'has_producer', 'producer'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('movie', 'has_writer', 'writer'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('movie', 'similar', 'movie'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('user', 'rates', 'movie'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True),
            ('user', 'similar', 'user'): GATConv(embedding_dim, embedding_dim, num_heads=num_heads, allow_zero_in_degree=True)
        })

        # 用于处理拼接特征的附加层
        self.fc = nn.Sequential(
            nn.Linear(32 * 3 * num_heads + embedding_dim, 128),
            nn.LeakyReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(128, 128)
        )

    def forward(self, user_ids, pos_item_ids, neg_item_ids, graph, features):
        print(f"Forward method called with parameters: {locals()}")

        user_emb = self.user_embeddings(user_ids)
        pos_item_emb = self.item_embeddings(pos_item_ids)
        neg_item_emb = self.item_embeddings(neg_item_ids)

        if graph is not None and features is not None:
            image_features = F.leaky_relu(self.image_transform(features['movieimage']))
            text_features = F.leaky_relu(self.text_transform(features['movietext']))
            audio_features = F.leaky_relu(self.audio_transform(features['movieaudio']))

            # 确保特征数量与节点数量一致
            print(f"Image features shape: {image_features.shape}, Graph num nodes: {graph.number_of_nodes()}")
            image_features = self.gat_image1(graph, image_features)
            image_features = F.leaky_relu(image_features)
            image_features = F.dropout(image_features, p=0.2)
            image_features = self.gat_image2(graph, image_features).flatten(1)

            print(f"Text features shape: {text_features.shape}, Graph num nodes: {graph.number_of_nodes()}")
            text_features = self.gat_text1(graph, text_features)
            text_features = F.leaky_relu(text_features)
            text_features = F.dropout(text_features, p=0.2)
            text_features = self.gat_text2(graph, text_features).flatten(1)

            print(f"Audio features shape: {audio_features.shape}, Graph num nodes: {graph.number_of_nodes()}")
            audio_features = self.gat_audio1(graph, audio_features)
            audio_features = F.leaky_relu(audio_features)
            audio_features = F.dropout(audio_features, p=0.2)
            audio_features = self.gat_audio2(graph, audio_features).flatten(1)

            with graph.local_scope():
                h = self.hetero_conv(graph, {
                    'movie': torch.cat([pos_item_emb, neg_item_emb], dim=0),
                    'user': user_emb,
                    'actor': self.embeddings['actor'].weight,
                    'actress': self.embeddings['actress'].weight,
                    'composer': self.embeddings['composer'].weight,
                    'director': self.embeddings['director'].weight,
                    'editor': self.embeddings['editor'].weight,
                    'producer': self.embeddings['producer'].weight,
                    'writer': self.embeddings['writer'].weight
                })

                movie_features = h['movie']
                combined_features = torch.cat((image_features, text_features, audio_features, movie_features), dim=1)
                combined_features = F.leaky_relu(self.fc(combined_features))

            return user_emb, pos_item_emb, combined_features

        return user_emb, pos_item_emb, None


In [62]:

def bpr_loss(users, pos_items, neg_items, lambda_reg, model):
    # 获取用户和项目的嵌入
    user_embeddings = model.embedding_user(users)
    pos_item_embeddings = model.embedding_item(pos_items)
    neg_item_embeddings = model.embedding_item(neg_items)
    
    # 计算用户对正样本和负样本的偏好预测
    pos_scores = torch.sum(user_embeddings * pos_item_embeddings, dim=1)
    neg_scores = torch.sum(user_embeddings * neg_item_embeddings, dim=1)
    
    # 使用 logsigmoid 提高数值稳定性
    loss = -torch.mean(F.logsigmoid(pos_scores - neg_scores))
    
    # 添加 L2 正则化
    reg_loss = lambda_reg * (user_embeddings.norm(p=2).pow(2) + 
                             pos_item_embeddings.norm(p=2).pow(2) +
                             neg_item_embeddings.norm(p=2).pow(2))
    
    return loss + reg_loss


評估函數

In [63]:
import torch
def dcg_at_k(scores, k=10):
    ranks = torch.log2(torch.arange(2, k+2).float()).to(scores.device)  # Log term in DCG formula
    return (scores[:k] / ranks).sum()  # Only consider the top k scores


def ndcg_at_k(predicted_scores, true_relevance, k=5):
    _, indices = torch.sort(predicted_scores, descending=True)
    true_sorted_by_pred = true_relevance[indices]
    ideal_sorted, _ = torch.sort(true_relevance, descending=True)

    dcg = dcg_at_k(true_sorted_by_pred[:k])
    idcg = dcg_at_k(ideal_sorted[:k])
    return (dcg / idcg).item() if idcg > 0 else 0.0
def recall_at_k(predicted_scores, true_labels, k):
    _, indices = torch.sort(predicted_scores, descending=True)
    true_sorted_by_pred = true_labels[indices]
    return true_sorted_by_pred[:k].float().sum().item() / true_labels.float().sum().item()


In [64]:
def create_graph_and_features(user_ids, pos_item_ids, neg_item_ids, device):
    user_ids = user_ids.to(device)
    pos_item_ids = pos_item_ids.to(device)
    neg_item_ids = neg_item_ids.to(device)

    # Ensure indices are unique and continuous across the whole graph
    all_users = torch.cat([user_ids, user_ids])  # Users are duplicated for pos and neg connections
    all_items = torch.cat([pos_item_ids, neg_item_ids])  # All item connections

    # Create mappings for user and item IDs to continuous indices
    unique_ids, inverse_indices = torch.unique(torch.cat([user_ids, all_items]), return_inverse=True)
    user_indices = inverse_indices[:len(user_ids)]
    item_indices = inverse_indices[len(user_ids):]

    # Create the graph
    src = torch.cat([user_indices, user_indices])  # Users for both pos and neg items
    dst = item_indices  # Combined pos and neg items
    g = dgl.graph((src, dst), num_nodes=unique_ids.numel()).to(device)

    # Add self-loops to the graph
    g = dgl.add_self_loop(g)

    # Assign features - assuming random features for simplicity
    g.ndata['feat'] = torch.randn(g.number_of_nodes(), 16).to(device)

    # Ensure the number of item features matches the number of unique item nodes
    num_items = g.number_of_nodes() - len(user_ids)
    features = {
        'movieimage': torch.randn(num_items, 2048).to(device),
        'movietext': torch.randn(num_items, 384).to(device),
        'movieaudio': torch.randn(num_items, 128).to(device)
    }

    print(f"Graph has {g.number_of_nodes()} nodes")
    for key, value in features.items():
        print(f"Feature {key} has shape {value.shape}")

    return g, features


In [65]:
def train_model(model, train_loader, epochs, lambda_reg, optimizer, device, validation_loader=None):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            user_ids, pos_item_ids, neg_item_ids = data
            user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)
            
            # 创建图结构和特征
            graph, features = create_graph_and_features(user_ids, pos_item_ids, neg_item_ids, device)

            # 输出调试信息
            print(f"Epoch {epoch+1}:")
            print(f"user_ids: {user_ids.shape}, pos_item_ids: {pos_item_ids.shape}, neg_item_ids: {neg_item_ids.shape}")
            print(f"graph: {graph}")
            print(f"features: {[key for key in features.keys()]}")

            optimizer.zero_grad()
            user_embeddings, pos_item_embeddings, combined_features = model(user_ids, pos_item_ids, neg_item_ids, graph, features)
            
            neg_item_embeddings = combined_features if combined_features is not None else pos_item_embeddings  # 使用 combined_features 作為負樣本嵌入

            # 计算损失
            loss = bpr_loss(user_embeddings, pos_item_embeddings, neg_item_embeddings, lambda_reg)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch {epoch+1}: Average Training Loss: {total_loss / len(train_loader)}')

        if validation_loader:
            evaluate_metrics(model, validation_loader, device)


In [66]:

def evaluate_metrics(model, validation_loader, device, k=5):
    device = next(model.parameters()).device  # 获取模型参数所在的设备
    model.eval()
    total_dcg = 0
    total_ndcg = 0
    total_recall = 0
    total_relevant_items = 0

    with torch.no_grad():
        for data in validation_loader:
            user_ids, pos_item_ids, neg_item_ids = data
            user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)
            
            # 创建图结构和特征
            graph, features = create_graph_and_features(user_ids, pos_item_ids, neg_item_ids, device)
            
            # 前向传播
            user_embeddings, pos_item_embeddings, combined_features = model(user_ids, pos_item_ids, graph, features)
            neg_item_embeddings = combined_features  # 使用combined_features作为负样本嵌入
            
            # 计算评分
            pos_scores = (user_embeddings * pos_item_embeddings).sum(dim=1)
            neg_scores = (user_embeddings * neg_item_embeddings).sum(dim=1)
            
            scores = torch.cat((pos_scores, neg_scores))
            labels = torch.cat((torch.ones_like(pos_scores), torch.zeros_like(neg_scores)))  # 真实标签
            
            # 计算分数并排序
            _, indices = torch.sort(scores, descending=True)
            sorted_labels = labels[indices]
            
            # 使用已定义的函数计算 DCG 和 NDCG
            dcg_value = dcg_at_k(sorted_labels, k)
            idcg_value = dcg_at_k(torch.ones(k).to(device), k)  # 理想情况下的排序
            ndcg_value = dcg_value / max(idcg_value, 1e-10)  # 避免除以零
            recall_value = sorted_labels[:k].sum() / labels.sum()

            total_dcg += dcg_value
            total_ndcg += ndcg_value
            total_recall += recall_value
            total_relevant_items += labels.sum()

    average_dcg = total_dcg / len(validation_loader)
    average_ndcg = total_ndcg / len(validation_loader)
    average_recall = total_recall / len(validation_loader)
    print(f'Average DCG@{k}: {average_dcg:.4f}')
    print(f'Average NDCG@{k}: {average_ndcg:.4f}')
    print(f'Average Recall@{k}: {average_recall:.4f}')

In [67]:

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_sizes = {
    'movie': 2560,    # 如果电影节点存储所有多媒体特征的总和
    'actor': 4566,      # 演员的嵌入维度
    'actress': 3701,    # 女演员的嵌入维度
    'director': 4010,   # 导演的嵌入维度
    'producer': 3952,   # 制片的嵌入维度
    'movieimage': 2048,  # 图像特征维度
    'movietext': 384,    # 文字特征维度
    'movieaudio': 128,   # 音频特征维度
    'user': 12171,        # 假设用戶的嵌入维度，可能需要根据用户数量和系统复杂性进行调整
    'composer': 512,
    'editor':512,
    'writer':512
}

In [68]:

# 初始化和配置模型，加载数据，然后开始训练和评估
model = EnhancedHeteroGraphConv(num_users, num_items, feature_sizes)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
train_model(model, train_loader,15, 0.01, optimizer, device, valid_loader)

# 保存模型
torch.save(model.state_dict(), 'D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth')

# 加载模型
model.load_state_dict(torch.load('D:\CODE\multi-model knowledge graph multi-graph recommendation system\code\data\model.pth'))
model.eval()


Graph has 358 nodes
Feature movieimage has shape torch.Size([230, 2048])
Feature movietext has shape torch.Size([230, 384])
Feature movieaudio has shape torch.Size([230, 128])
Epoch 1:
user_ids: torch.Size([128]), pos_item_ids: torch.Size([128]), neg_item_ids: torch.Size([128])
graph: Graph(num_nodes=358, num_edges=614,
      ndata_schemes={'feat': Scheme(shape=(16,), dtype=torch.float32)}
      edata_schemes={})
features: ['movieimage', 'movietext', 'movieaudio']
Forward method called with parameters: {'self': EnhancedHeteroGraphConv(
  (user_embeddings): Embedding(12171, 128)
  (item_embeddings): Embedding(5996, 128)
  (embeddings): ModuleDict(
    (actor): Embedding(10000, 16)
    (actress): Embedding(10000, 16)
    (director): Embedding(10000, 16)
    (producer): Embedding(10000, 16)
    (user): Embedding(10000, 16)
    (composer): Embedding(10000, 16)
    (editor): Embedding(10000, 16)
    (writer): Embedding(10000, 16)
  )
  (image_transform): Linear(in_features=2048, out_feature

DGLError: Expect number of features to match number of nodes (len(u)). Got 230 and 358 instead.

5/11測試

In [ ]:
def generate_rankings_and_ground_truth(model, data_loader, device):
    model.eval()
    rank_list = {}
    ground_truth = {}
    with torch.no_grad():
        for batch in data_loader:
            user_ids, pos_item_ids, neg_item_ids = batch
            user_ids, pos_item_ids, neg_item_ids = user_ids.to(device), pos_item_ids.to(device), neg_item_ids.to(device)

            # 调用模型
            pos_outputs = model(user_ids, pos_item_ids)
            neg_outputs = model(user_ids, neg_item_ids)

            # 根据输出选择得分计算方式
            if len(pos_outputs) == 3:
                pos_scores = (pos_outputs[0] * pos_outputs[2]).sum(dim=1)
                neg_scores = (neg_outputs[0] * neg_outputs[2]).sum(dim=1)
            else:
                pos_scores = (pos_outputs[0] * pos_outputs[1]).sum(dim=1)
                neg_scores = (neg_outputs[0] * neg_outputs[1]).sum(dim=1)

            scores = torch.cat((pos_scores, neg_scores), dim=0)
            all_item_ids = torch.cat((pos_item_ids, neg_item_ids), dim=0)
            _, indices = torch.sort(scores, descending=True)
            sorted_items = all_item_ids[indices]

            for user_id, item in zip(user_ids, sorted_items):
                user = user_id.item()
                item = item.item() if hasattr(item, 'item') else item  # 确保item是单个元素
                if user not in rank_list:
                    rank_list[user] = []
                rank_list[user].append(item)

                if user not in ground_truth:
                    ground_truth[user] = set()
                ground_truth[user].update(pos_item_ids.tolist())

    return rank_list, ground_truth
